# Imports and helper functions

To import packages and modules to Jupyter notebook, you need to setup a conda environment. Here we call it `gpst`.
```
conda create --name gpst
conda install pypsa pandapower jupyterlab
pip install yaml vresutils==0.3.1
```
Upgrade to pandapower to develop branch
```
pip install git+git://github.com:e2nIEE/pandapower@develop
```
To  add the kernel for the jupyter notebook
```
pip install ipykernel
ipython kernel install --user --name=gpst
```

Open the jupyter lab notebook by typing `jupyter lab` in the terminal.


In [1]:
import os
import timeit
import pandas as pd

import numpy as np
import pandapower as pp
import pandapower.converter
        
import logging

# Show all pandas columns in jupyter
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
# Optional. Take local PyPSA-Dev installation to make adjustments
pypsa_path = os.getcwd()+"/PyPSA"  # require to have `PyPSA` cloned in ~/power-flow-exercise/example_pandapower/<PyPSA>`
import sys
sys.path.insert(0, f"{pypsa_path}")

In [3]:
import pypsa

In [4]:
def _sets_path_to_root(root_directory_name):
    """
    Search and sets path to the given root directory (root/path/file).

    Parameters
    ----------
    root_directory_name : str
        Name of the root directory.
    n : int
        Number of folders the function will check upwards/root directed.

    """
    import os

    repo_name = root_directory_name
    n = 8  # check max 8 levels above. Random default.
    n0 = n

    while n >= 0:
        n -= 1
        # if repo_name is current folder name, stop and set path
        if repo_name == os.path.basename(os.path.abspath(".")):
            repo_path = os.getcwd()  # os.getcwd() = current_path
            os.chdir(repo_path)  # change dir_path to repo_path
            print("This is the repository path: ", repo_path)
            print("Had to go %d folder(s) up." % (n0 - 1 - n))
            break
        # if repo_name NOT current folder name for 5 levels then stop
        if n == 0:
            print("Cant find the repo path.")
        # if repo_name NOT current folder name, go one dir higher
        else:
            upper_path = os.path.dirname(
                os.path.abspath("."))  # name of upper folder
            os.chdir(upper_path)

In [5]:
_sets_path_to_root("power-flow-exercise")  # name of the git # n.generators_t.p_set = clone folder

This is the repository path:  /home/max/OneDrive/PHD-Flexibility/07_pypsa-africa/0github/power-flow-exercise
Had to go 1 folder(s) up.


In [6]:
# Set logger
logging.basicConfig(filename=os.path.join("example_pandapower", "log.out"),
                    filemode='w',
                    format='%(asctime)s %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

# Pandapower import of Matpower

In [7]:
net = pp.converter.from_mpc(os.path.join("reference-matpower", "RTS_GMLC.mat"))

# Comparing Pandapower to solved Matpower network

In [8]:
def compare_to_matpower(net):
    """
    Compares pandapower network to matpower network.
    """
    bus_results = pd.read_csv(os.path.join("reference-matpower", "results", "bus.csv"), index_col=0)
    branch_results = pd.read_csv(os.path.join("reference-matpower", "results", "flow.csv"), index_col=0)

    # compare bus results
    # merged_results = pd.merge(bus_results, net.res_bus, how='inner', left_index=True, right_on=net.bus.id)
    merged_results = pd.merge(bus_results, net.res_bus, how='inner', left_index=True, right_on=net.bus.name)
    merged_results['diff_vm_pu'] = merged_results.v_mag - merged_results.vm_pu
    merged_results['diff_va_degree'] = merged_results.v_ang - merged_results.va_degree
    logger.info(f"\n{merged_results[['diff_vm_pu', 'diff_va_degree']].describe()}")

    # compare branch results
    merged_results_line = pd.merge(branch_results, net.res_line, how='inner', left_index=True, right_index=True)
    merged_results_line['diff_from_p_mw'] = merged_results_line.p_from_mw - merged_results_line.from_bus_inj_p
    merged_results_line['diff_to_p_mw'] = merged_results_line.p_to_mw - merged_results_line.to_bus_inj_p
    merged_results_line['diff_from_q_mvar'] = merged_results_line.q_from_mvar - merged_results_line.from_bus_inj_q
    merged_results_line['diff_to_q_mvar'] = merged_results_line.q_to_mvar - merged_results_line.to_bus_inj_q
    merged_results_line['diff_loss_p'] = merged_results_line.pl_mw - merged_results_line.loss_p
    cols = ['diff_from_p_mw', 'diff_to_p_mw', 'diff_from_q_mvar', 'diff_to_q_mvar', 'diff_loss_p']
    logger.info(f"\n{merged_results_line[cols].describe()}")

In [9]:
net.sgen.drop(net.sgen[~net.sgen.in_service].index, inplace=True)  # TODO: check if this is correct (dropping not in_service elements from net.sgen
pp.runpp(net)
compare_to_matpower(net)  # Info: Will be saved in logger 'log.out'

# Prepare Pandapower network

In [10]:
def make_name_to_index(net):
    """
    Makes name as index.
    
    PyPSA requires a unique name as index.
    """
    for elm in net.keys():
        if elm.startswith("_") or not isinstance(net[elm], pd.DataFrame) or len(net[elm]) == 0:
            continue
        # let's keep the bus names from mpc
        if elm == 'bus':
            continue
        net[elm]['name'] = [f"{elm}_{i}" for i in net[elm].index.values]
        # checking because in_service not supported by pypsa
        if 'in_service' in net[elm].columns:
            assert np.alltrue(net[elm]['in_service']), f"not in_service elements found in {elm}"
        # checking because switches are not supported by pypsa
        if elm == 'switch':
            assert np.alltrue(net[elm]['closed']), "open switches found"
        if elm == "trafo3w" and len(net[elm]) > 0:
            logger.warning("found trafo3w in net (not supported by pypsa converter!!!)")
        if elm == "shunt" and len(net[elm]) > 0:
            logger.warning("found shunt in net (not supported by pypsa converter!!!)")

In [11]:
make_name_to_index(net)

# PyPSA import of Pandapower network

In [12]:
# Build empty PyPSA network
network = pypsa.Network()
# Import pandapower
network.import_from_pandapower_net(net, True, use_pandapower_index=True)

### Checks if import is correct

In [13]:
# now let's check some basic infos
assert len(network.buses) == len(net.bus)
assert len(network.generators) == (len(net.gen) + len(net.sgen) + len(net.ext_grid))
assert len(network.loads) == len(net.load)
assert len(network.transformers) == len(net.trafo)
# todo: shunt impedances are not supported
# todo: trafo tap positions are not supported
# todo: trafo3w are not supported

Pandapower

In [14]:
net.trafo

,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,i0_percent,shift_degree,tap_side,tap_neutral,tap_min,tap_max,tap_step_percent,tap_step_degree,tap_pos,tap_phase_shifter,parallel,df,in_service,pt_percent,oltc,max_loading_percent
0,0,None,23,2,400.0,230.0,138.0,33.609522,0.8,0.0,-0.0,0.0,lv,0,NaN,NaN,1.5,NaN,1,False,1,1.0,True,NaN,False,100.0
1,1,None,10,8,400.0,230.0,138.0,33.609522,0.8,0.0,-0.0,0.0,lv,0,NaN,NaN,3.0,NaN,1,False,1,1.0,True,NaN,False,100.0
2,2,None,11,8,400.0,230.0,138.0,33.609522,0.8,0.0,-0.0,0.0,lv,0,NaN,NaN,3.0,NaN,1,False,1,1.0,True,NaN,False,100.0
3,3,None,10,9,400.0,230.0,138.0,33.609522,0.8,0.0,-0.0,0.0,lv,0,NaN,NaN,1.5,NaN,1,False,1,1.0,True,NaN,False,100.0
4,4,None,11,9,400.0,230.0,138.0,33.609522,0.8,0.0,-0.0,0.0,lv,0,NaN,NaN,1.5,NaN,1,False,1,1.0,True,NaN,False,100.0
5,5,None,47,26,400.0,230.0,138.0,33.609522,0.8,0.0,-0.0,0.0,lv,0,NaN,NaN,1.5,NaN,1,False,1,1.0,True,NaN,False,100.0
6,6,None,34,32,400.0,230.0,138.0,33.609522,0.8,0.0,-0.0,0.0,lv,0,NaN,NaN,3.0,NaN,1,False,1,1.0,True,NaN,False,100.0
7,7,None,35,32,400.0,230.0,138.0,33.609522,0.8,0.0,-0.0,0.0,lv,0,NaN,NaN,3.0,NaN,1,False,1,1.0,True,NaN,False,100.0
8,8,None,34,33,400.0,230.0,138.0,33.609522,0.8,0.0,-0.0,0.0,lv,0,NaN,NaN,1.5,NaN,1,False,1,1.0,True,NaN,False,100.0
9,9,None,35,33,400.0,230.0,138.0,33.609522,0.8,0.0,-0.0,0.0,lv,0,NaN,NaN,1.5,NaN,1,False,1,1.0,True,NaN,False,100.0


In [15]:
net.sn_mva ## PyPSA needs to deal with the different base!!! - Maybe ignore it. 

100

# Inspect PyPSA network before solving

In [16]:
n = network

In [17]:
n.snapshots

Index(['now'], dtype='object', name='snapshot')

In [18]:
n.global_constraints

attribute,type,investment_period,carrier_attribute,sense,constant,mu
GlobalConstraint,,,,,,


In [20]:
n.buses

,v_nom,v_mag_pu_set,type,x,y,carrier,unit,v_mag_pu_min,v_mag_pu_max,control,sub_network
Bus,,,,,,,,,,,
0,138.0,1.0468,,0.0,0.0,AC,None,0.0,inf,PQ,
1,138.0,1.0467,,0.0,0.0,AC,None,0.0,inf,PQ,
2,138.0,1.0000,,0.0,0.0,AC,None,0.0,inf,PQ,
3,138.0,1.0000,,0.0,0.0,AC,None,0.0,inf,PQ,
4,138.0,1.0000,,0.0,0.0,AC,None,0.0,inf,PQ,
5,138.0,1.0000,,0.0,0.0,AC,None,0.0,inf,PQ,
6,138.0,1.0500,,0.0,0.0,AC,None,0.0,inf,PQ,
7,138.0,1.0000,,0.0,0.0,AC,None,0.0,inf,PQ,
8,138.0,1.0000,,0.0,0.0,AC,None,0.0,inf,PQ,


In [21]:
n.generators

,p_set,q_set,bus,control,type,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,sign,carrier,marginal_cost,build_year,lifetime,capital_cost,efficiency,committable,start_up_cost,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,8.00,0.000000,0,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
1,8.00,0.000000,1,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
2,355.00,0.000000,6,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
3,5.00,0.000000,14,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
4,155.00,0.000000,15,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
5,355.00,0.000000,17,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
6,155.00,0.000000,22,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
7,8.00,0.000000,24,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
8,8.00,0.000000,25,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0


In [32]:
n.loads_t

{'p_set': Empty DataFrame
 Columns: []
 Index: [now],
 'q_set': Empty DataFrame
 Columns: []
 Index: [now],
 'p': Load          0     1      2     3     4      5      6      7      8      9  \
 snapshot                                                                      
 now       108.0  97.0  180.0  74.0  71.0  136.0  125.0  171.0  175.0  195.0   
 
 Load         10     11     12     13     14     15     16     17    18     19  \
 snapshot                                                                        
 now       265.0  194.0  317.0  100.0  333.0  181.0  128.0  108.0  97.0  180.0   
 
 Load        20    21     22     23     24     25     26     27     28     29  \
 snapshot                                                                       
 now       74.0  71.0  136.0  125.0  171.0  175.0  195.0  265.0  194.0  317.0   
 
 Load         30     31     32     33     34    35     36    37    38     39  \
 snapshot                                                               

In [23]:
n.loads

,p_set,q_set,bus,carrier,type,sign
Load,,,,,,
0,108.0,22.0,0,,,-1.0
1,97.0,20.0,1,,,-1.0
2,180.0,37.0,2,,,-1.0
3,74.0,15.0,3,,,-1.0
4,71.0,14.0,4,,,-1.0
5,136.0,28.0,5,,,-1.0
6,125.0,25.0,6,,,-1.0
7,171.0,35.0,7,,,-1.0
8,175.0,36.0,8,,,-1.0


In [33]:
n.lines

,r,x,b,s_nom,bus0,bus1,length,num_parallel,type,g,s_nom_extendable,s_nom_min,s_nom_max,s_max_pu,capital_cost,build_year,lifetime,carrier,terrain_factor,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt,v_nom
Line,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.57132,2.66616,0.002421,101.036297,0,1,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,AC,1.0,-inf,inf,0,0.00014,0.000030,0.0,46.1,0.00014,0.000030,0.0,138.0
1,10.47420,40.18284,0.000299,101.036297,0,2,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,AC,1.0,-inf,inf,0,0.00211,0.000550,0.0,5.7,0.00211,0.000550,0.0,138.0
2,4.18968,16.18740,0.000121,101.036297,0,4,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,AC,1.0,-inf,inf,0,0.00085,0.000220,0.0,2.3,0.00085,0.000220,0.0,138.0
3,6.28452,24.18588,0.000179,101.036297,1,3,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,AC,1.0,-inf,inf,0,0.00127,0.000330,0.0,3.4,0.00127,0.000330,0.0,138.0
4,9.52200,36.56448,0.000273,101.036297,1,5,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,AC,1.0,-inf,inf,0,0.00192,0.000500,0.0,5.2,0.00192,0.000500,0.0,138.0
5,5.90364,22.66236,0.000168,101.036297,2,8,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,AC,1.0,-inf,inf,0,0.00119,0.000310,0.0,3.2,0.00119,0.000310,0.0,138.0
6,5.14188,19.80576,0.000147,101.036297,3,8,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,AC,1.0,-inf,inf,0,0.00104,0.000270,0.0,2.8,0.00104,0.000270,0.0,138.0
7,4.38012,16.75872,0.000126,101.036297,4,9,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,AC,1.0,-inf,inf,0,0.00088,0.000230,0.0,2.4,0.00088,0.000230,0.0,138.0
8,2.66616,11.61684,0.012912,101.036297,5,9,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,AC,1.0,-inf,inf,0,0.00061,0.000140,0.0,245.9,0.00061,0.000140,0.0,138.0


In [34]:
n.transformers

,phase_shift,s_nom,bus0,bus1,r,x,g,b,tap_position,type,model,s_nom_extendable,s_nom_min,s_nom_max,s_max_pu,capital_cost,num_parallel,tap_ratio,tap_side,build_year,lifetime,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Transformer,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.4,23,2,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,-0.0,0.0021,0.00005,0.0
1,0.0,0.4,10,8,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,-0.0,0.0021,0.00005,0.0
2,0.0,0.4,11,8,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,-0.0,0.0021,0.00005,0.0
3,0.0,0.4,10,9,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,-0.0,0.0021,0.00005,0.0
4,0.0,0.4,11,9,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,-0.0,0.0021,0.00005,0.0
5,0.0,0.4,47,26,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,-0.0,0.0021,0.00005,0.0
6,0.0,0.4,34,32,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,-0.0,0.0021,0.00005,0.0
7,0.0,0.4,35,32,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,-0.0,0.0021,0.00005,0.0
8,0.0,0.4,34,33,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,-0.0,0.0021,0.00005,0.0


In [ ]:
n.transformers_t.p1.T.plot()

In [35]:
n.links

attribute,bus0,bus1,type,carrier,efficiency,build_year,lifetime,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_set,p_min_pu,p_max_pu,capital_cost,marginal_cost,length,terrain_factor,p_nom_opt
Link,,,,,,,,,,,,,,,,,,,


In [36]:
n.stores

attribute,bus,type,carrier,e_nom,e_nom_extendable,e_nom_min,e_nom_max,e_min_pu,e_max_pu,e_initial,e_initial_per_period,e_cyclic,e_cyclic_per_period,p_set,q_set,sign,marginal_cost,capital_cost,standing_loss,build_year,lifetime,e_nom_opt
Store,,,,,,,,,,,,,,,,,,,,,,


In [37]:
n.storage_units

attribute,bus,control,type,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,q_set,sign,carrier,marginal_cost,capital_cost,build_year,lifetime,state_of_charge_initial,state_of_charge_initial_per_period,state_of_charge_set,cyclic_state_of_charge,cyclic_state_of_charge_per_period,max_hours,efficiency_store,efficiency_dispatch,standing_loss,inflow,p_nom_opt
StorageUnit,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [38]:
n.shunt_impedances

attribute,bus,g,b,sign,g_pu,b_pu,v_nom
ShuntImpedance,,,,,,,


In [ ]:
n.carriers

In [ ]:
n.links

# Adjustment of pandapower network

In [ ]:
n.generators.loc[:,"p_set"] = abs(n.generators.loc[:,"p_set"])
n.generators.loc[:,"q_set"] = -(n.generators.loc[:,"q_set"])

In [ ]:
n.loads.loc[:,["p_set", "q_set"]] = (n.loads.loc[:,["p_set", "q_set"]])*0.5

In [ ]:
n.loads.loc[:,["q_set"]] = (n.loads.loc[:,["q_set"]])*0.1

In [ ]:
n.buses.v_mag_pu_set = 1

In [25]:
n.lines.loc["104","r"] = 1

In [26]:
n.transformers.r = 2e-5
n.transformers.x = 8.4e-4
n.transformers.tap_position = 0
n.transformers.r = 2e-5
n.transformers.x = 8.4e-4
n.transformers

,phase_shift,s_nom,bus0,bus1,r,x,g,b,tap_position,type,model,s_nom_extendable,s_nom_min,s_nom_max,s_max_pu,capital_cost,num_parallel,tap_ratio,tap_side,build_year,lifetime,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Transformer,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.4,23,2,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.84,0.02,0.0,-0.0,0.84,0.02,0.0
1,0.0,0.4,10,8,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.84,0.02,0.0,-0.0,0.84,0.02,0.0
2,0.0,0.4,11,8,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.84,0.02,0.0,-0.0,0.84,0.02,0.0
3,0.0,0.4,10,9,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.84,0.02,0.0,-0.0,0.84,0.02,0.0
4,0.0,0.4,11,9,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.84,0.02,0.0,-0.0,0.84,0.02,0.0
5,0.0,0.4,47,26,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.84,0.02,0.0,-0.0,0.84,0.02,0.0
6,0.0,0.4,34,32,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.84,0.02,0.0,-0.0,0.84,0.02,0.0
7,0.0,0.4,35,32,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.84,0.02,0.0,-0.0,0.84,0.02,0.0
8,0.0,0.4,34,33,0.00002,0.00084,0.0,-0.0,0,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.84,0.02,0.0,-0.0,0.84,0.02,0.0


In [ ]:
n.buses_t.v_mag_pu.T.plot()

In [ ]:
n.sub_networks

In [ ]:
pandanet = n.copy()

In [ ]:
pandanet.buses

In [ ]:
n.mremove("Load", pandanet.loads.index.values)

In [ ]:
n.madd("Load", pandanet.loads.index.values,

       bus=pandanet.loads.bus.values,

       p_set=pandanet.loads.p_set.values*0.5)

In [ ]:
n.loads_t.p_set = n.loads_t.p

In [ ]:
n.generators_t.p_set = n.generators_t.p

In [ ]:
n.loads

# Solve network

In [ ]:
n.loads.p_set.sum()
n.loads.q_set.sum()

n.transformers_t.p0.sum().plot()

In [ ]:
n.buses.v_mag_pu_set = 1.0

In [27]:
#n.lopf()
#n.pf(use_seed=True, distribute_slack=True)
#n.pf(x_tol=1e-6,)
n.pf()
# n.lpf()

{'n_iter': SubNetwork  0
 snapshot     
 now         5,
 'error': SubNetwork             0
 snapshot                
 now         1.905586e-10,
 'converged': SubNetwork     0
 snapshot        
 now         True}

In [31]:
n.loads

,p_set,q_set,bus,carrier,type,sign
Load,,,,,,
0,108.0,22.0,0,,,-1.0
1,97.0,20.0,1,,,-1.0
2,180.0,37.0,2,,,-1.0
3,74.0,15.0,3,,,-1.0
4,71.0,14.0,4,,,-1.0
5,136.0,28.0,5,,,-1.0
6,125.0,25.0,6,,,-1.0
7,171.0,35.0,7,,,-1.0
8,175.0,36.0,8,,,-1.0


In [ ]:
n.generators_t

In [ ]:
n.generators.groupby("control")[["p_set", "q_set"]].sum()

In [ ]:
n.generators.drop(n.generators.loc[n.generators["control"]=="PQ"].index, inplace=True)

In [39]:
n.generators

,p_set,q_set,bus,control,type,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,sign,carrier,marginal_cost,build_year,lifetime,capital_cost,efficiency,committable,start_up_cost,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,8.00,0.000000,0,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
1,8.00,0.000000,1,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
2,355.00,0.000000,6,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
3,5.00,0.000000,14,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
4,155.00,0.000000,15,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
5,355.00,0.000000,17,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
6,155.00,0.000000,22,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
7,8.00,0.000000,24,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
8,8.00,0.000000,25,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0


In [ ]:
n.loads[["p_set", "q_set"]].sum()

# Planning Exercise Test

### Load cost.csv

In [40]:
import yaml
from vresutils.costdata import annuity

In [41]:
def load_costs(Nyears=1., tech_costs=None, config=None, elec_config=None):
    if tech_costs is None:
        tech_costs = os.getcwd()+"/example-pypsa/costs.csv"

    if config is None:
        ### Loads raw config.yaml
        path = os.getcwd()+"/example-pypsa/config.yaml"
        with open(path, "r") as stream:
            try:
                config = yaml.safe_load(stream)
            except yaml.YAMLError as exc:
                print(exc)
        config = config['costs']

    # set all asset costs and other parameters
    costs = pd.read_csv(tech_costs, index_col=list(range(3))).sort_index()

    # correct units to MW and EUR
    costs.loc[costs.unit.str.contains("/kW"),"value"] *= 1e3
    costs.loc[costs.unit.str.contains("USD"),"value"] *= config['USD2013_to_EUR2013']

    idx = pd.IndexSlice
    costs = (costs.loc[idx[:,config['year'],:], "value"]
             .unstack(level=2).groupby("technology").sum(min_count=1))

    costs = costs.fillna({"CO2 intensity" : 0,
                          "FOM" : 0,
                          "VOM" : 0,
                          "discount rate" : config['discountrate'],
                          "efficiency" : 1,
                          "fuel" : 0,
                          "investment" : 0,
                          "lifetime" : 25})

    costs["capital_cost"] = ((annuity(costs["lifetime"], costs["discount rate"]) +
                             costs["FOM"]/100.) *
                             costs["investment"] * Nyears)

    costs.at['OCGT', 'fuel'] = costs.at['gas', 'fuel']
    costs.at['CCGT', 'fuel'] = costs.at['gas', 'fuel']

    costs['marginal_cost'] = costs['VOM'] + costs['fuel'] / costs['efficiency']

    costs = costs.rename(columns={"CO2 intensity": "co2_emissions"})

    costs.at['OCGT', 'co2_emissions'] = costs.at['gas', 'co2_emissions']
    costs.at['CCGT', 'co2_emissions'] = costs.at['gas', 'co2_emissions']

    costs.at['solar', 'capital_cost'] = 0.5*(costs.at['solar-rooftop', 'capital_cost'] +
                                             costs.at['solar-utility', 'capital_cost'])

    return costs

In [42]:
def add_nice_carrier_names(n, config=None):
    if config is None:
        path = os.getcwd()+"/example-pypsa/config.yaml"
        with open(path, "r") as stream:
            try:
                config = yaml.safe_load(stream)
            except yaml.YAMLError as exc:
                print(exc)
        config = config

    carrier_i = n.carriers.index
    nice_names = (pd.Series(config['plotting']['nice_names'])
                  .reindex(carrier_i).fillna(carrier_i.to_series().str.title()))
    n.carriers['nice_name'] = nice_names
    colors = pd.Series(config['plotting']['tech_colors']).reindex(carrier_i)
    if colors.isna().any():
        missing_i = list(colors.index[colors.isna()])
        logger.warning(f'tech_colors for carriers {missing_i} not defined '
                       'in config.')
    n.carriers['color'] = colors

In [43]:
costs = load_costs()

In [44]:
costs

parameter,co2_emissions,FOM,VOM,discount rate,efficiency,fuel,investment,lifetime,capital_cost,marginal_cost
technology,,,,,,,,,,
CCGT,0.187,2.500000,4.00,0.07,0.500,21.6,800000.00,30.0,84469.122809,47.200000
DAC,0.000,4.000000,0.00,0.07,1.000,0.0,250.00,30.0,30.146601,0.000000
H2 pipeline,0.000,5.000000,0.00,0.07,0.980,0.0,267.00,40.0,33.377440,0.000000
HVAC overhead,0.000,2.000000,0.00,0.07,1.000,0.0,400.00,40.0,38.003656,0.000000
HVDC inverter pair,0.000,2.000000,0.00,0.07,1.000,0.0,150000.00,40.0,14251.370831,0.000000
HVDC overhead,0.000,2.000000,0.00,0.07,1.000,0.0,400.00,40.0,38.003656,0.000000
HVDC submarine,0.000,2.000000,0.00,0.07,1.000,0.0,2000.00,40.0,190.018278,0.000000
OCGT,0.187,3.750000,3.00,0.07,0.390,21.6,400000.00,30.0,47234.561404,58.384615
PHS,0.000,1.000000,0.00,0.07,0.750,0.0,2000000.00,80.0,160627.143522,0.000000


In [45]:
add_nice_carrier_names(n)

In [46]:
n.generators.loc[:,"carrier"] = "OCGT"
n.generators.loc[:,"capital_costs"] = costs.loc["OCGT", "capital_cost"]
n.generators.loc[:,"marginal_costs"] = costs.loc["OCGT", "marginal_cost"]
n.generators.loc[:,"lifetime"] = costs.loc["OCGT", "lifetime"]
n.generators.loc[:,"p_nom_extanable"] = "True"


n.lines.loc[:,'capital_cost'] = (n.lines['length'] * 1 * costs.at['HVAC overhead', 'capital_cost'])
n.lines.loc[:,"s_nom_extanable"] = "True"


n.transformers.loc[:,"s_nom_extanable"] = "True"


n.loads_t["p"] = n.loads_t["p"] * 5
n.loads["p_set"] = n.loads["p_set"] * 5

# Solve network

In [47]:
from pypsa.linopf import (get_var, define_constraints, linexpr, join_exprs,
                          network_lopf, ilopf)

In [48]:
### Loads raw config.yaml
path = os.getcwd()+"/example-pypsa/config.yaml"
with open(path, "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
config = config

In [49]:
def fix_bus_production(n, snapshots):
    total_demand = n.loads_t.p.sum().sum() 
    prod_per_bus = linexpr((1, get_var(n, 'Generator', 'p'))).groupby(n.generators.bus, axis=1).apply(join_exprs)
    define_constraints(n, prod_per_bus, '>=', total_demand/5, 'Bus', 'production_share')

def extra_functionality(n, snapshots):
    fix_bus_production(n, snapshots)

In [50]:
def solve_network(n, config, opts='', **kwargs):
    solver_options = config['solving']['solver'].copy()
    solver_name = solver_options.pop('name')
    cf_solving = config['solving']['options']
    track_iterations = cf_solving.get('track_iterations', False)
    min_iterations = cf_solving.get('min_iterations', 4)
    max_iterations = cf_solving.get('max_iterations', 6)

    # add to network for extra_functionality
    n.config = config
    n.opts = opts

    if cf_solving.get('skip_iterations', False):
        network_lopf(n, solver_name=solver_name, solver_options=solver_options,
                     extra_functionality=extra_functionality, **kwargs)
    else:
        ilopf(n, solver_name=solver_name, solver_options=solver_options,
              track_iterations=track_iterations,
              min_iterations=min_iterations,
              max_iterations=max_iterations,
              extra_functionality=extra_functionality, **kwargs)
    return n


In [51]:
from pathlib import Path

tmpdir = config['solving'].get('tmpdir')
if tmpdir is not None:
        Path(tmpdir).mkdir(parents=True, exist_ok=True)

n = solve_network(n, config=config,
                          solver_dir=tmpdir,)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-03-03
Read LP format model from file /tmp/pypsa-problem-oyckhsht.lp
Reading time = 0.01 seconds
obj: 586 rows, 217 columns, 1180 nonzeros
Set parameter Threads to value 4
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 1e-05
Set parameter AggFill to value 0
Set parameter PreDual to value 0
Set parameter GURO_PAR_BARDENSETHRESH to value 200
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 4 threads
Optimize a model with 586 rows, 217 columns and 1180 nonzeros
Model fingerprint: 0x2874f969
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-01, 9e+03]
Presolve removed 0 rows and 1 columns
Presolve time: 0.01s

Barrier solved model in 0 iterations and 0.01 seconds (0.00 work units)
Mod

AssertionError: Optimization failed with status warningand termination infeasible